# Descripción del Proyecto

Prepara un prototipo de un modelo de machine learning para Zyfra. La empresa desarrolla soluciones de eficiencia para la industria pesada.

El modelo debe predecir la cantidad de oro extraído del mineral de oro. Dispones de los datos de extracción y purificación.

El modelo ayudará a optimizar la producción y a eliminar los parámetros no rentables.

In [1]:
# Importacion de librerias a utilizar

import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import make_regression
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer

In [2]:
## Leemos los archivos

full = pd.read_csv('../source/gold_recovery_full.csv')
train = pd.read_csv('../source/gold_recovery_train.csv')
test = pd.read_csv('../source/gold_recovery_test.csv')

In [4]:
# Analizando cada DF
full.sample(5)

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
6177,2016-09-28 08:59:59,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,-799.623424,0.000000,-799.811909,-0.205237,-798.649826,0.636767,-800.004773,0.181404,-799.879522
3307,2016-05-31 18:59:59,3.613087,10.271419,8.871771,46.038007,58.770714,7.091629,2.503578,11.177440,2.815838,...,9.956235,-499.830923,7.962472,-502.566298,7.920568,-500.202079,5.971516,-500.709519,17.945298,-500.210361
4434,2016-07-17 17:59:59,6.273177,7.449815,15.068381,44.859492,73.605253,9.753937,2.061528,11.853678,2.548250,...,15.008931,-399.670577,5.019141,-400.619124,9.980482,-399.896836,5.004960,-400.142269,23.001797,-499.045602
17510,2018-01-13 13:59:59,5.557665,9.145988,8.393567,44.126672,62.441761,14.115016,3.885007,13.152438,5.075192,...,20.030104,-507.430398,15.013400,-499.990679,11.015693,-501.524058,9.009853,-499.808513,11.007642,-500.487556
1632,2016-03-23 00:00:00,4.107938,10.923634,4.856681,45.389809,63.846890,10.895997,2.023411,18.501445,3.542992,...,12.040359,-498.802169,11.958952,-500.329384,11.930437,-500.352691,10.007133,-500.457052,19.958651,-500.470024


In [5]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22716 entries, 0 to 22715
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                22716 non-null  object 
 1   final.output.concentrate_ag                         22627 non-null  float64
 2   final.output.concentrate_pb                         22629 non-null  float64
 3   final.output.concentrate_sol                        22331 non-null  float64
 4   final.output.concentrate_au                         22630 non-null  float64
 5   final.output.recovery                               20753 non-null  float64
 6   final.output.tail_ag                                22633 non-null  float64
 7   final.output.tail_pb                                22516 non-null  float64
 8   final.output.tail_sol                               22445 non-null  float64


In [7]:
full.columns

Index(['date', 'final.output.concentrate_ag', 'final.output.concentrate_pb',
       'final.output.concentrate_sol', 'final.output.concentrate_au',
       'final.output.recovery', 'final.output.tail_ag', 'final.output.tail_pb',
       'final.output.tail_sol', 'final.output.tail_au',
       'primary_cleaner.input.sulfate', 'primary_cleaner.input.depressant',
       'primary_cleaner.input.feed_size', 'primary_cleaner.input.xanthate',
       'primary_cleaner.output.concentrate_ag',
       'primary_cleaner.output.concentrate_pb',
       'primary_cleaner.output.concentrate_sol',
       'primary_cleaner.output.concentrate_au',
       'primary_cleaner.output.tail_ag', 'primary_cleaner.output.tail_pb',
       'primary_cleaner.output.tail_sol', 'primary_cleaner.output.tail_au',
       'primary_cleaner.state.floatbank8_a_air',
       'primary_cleaner.state.floatbank8_a_level',
       'primary_cleaner.state.floatbank8_b_air',
       'primary_cleaner.state.floatbank8_b_level',
       'primary_clean